In [1]:
# !unzip Registered.zip # unzip your file of images here. 

# update box credentials, imgs_per_line and desired_views before running. 
# tweak the cropping in the load_transform and in the crop_lightfield functions. 

In [2]:
from pathlib import Path
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import datetime
import tqdm
import cv2
import time
import os
import sys
import shutil

from lightfield_utils import load_lightfield_from_paths, resize_lightfield, load_lightfield, save_rendered_views
from lightfield_utils import pad_to_shape
from lightfield_canvas import DisplayLF

%load_ext autoreload
%autoreload 2

In [3]:
#configure: images per line, desired views
desired_views = [144, 192]
desired_resolution = (880, 660)

In [4]:
dst = Path("raw_benton_lightfield2021_05_06_13_11_.npy")

saved_lf = load_lightfield(dst)
DisplayLF(saved_lf, width= 572, height= 440).show()

Loaded lightfield with dtype: uint8 and shape: (21, 31, 630, 879, 3)


In [5]:
def pad_to_aspect_ratio(lf, aspect_ratio):
    """
    Pads lightfield to specified aspect ratio (width/height)
    """
    width, height = lf.shape[-2], lf.shape[-3]
    curr_aspect_ratio = width/height
    if curr_aspect_ratio > aspect_ratio: #current has excess width
        new_width = width
        new_height = int(width//aspect_ratio)
        bound_left, bound_right = 0, width
        bound_up = (new_height - height)//2
        bound_down = bound_up + height
    elif curr_aspect_ratio < aspect_ratio: #current aspect ratio has excess height:
        new_width = int(aspect_ratio*height)
        new_height = height
        bound_up, bound_down = 0, height
        bound_left = (new_width - width) //2
        bound_right = bound_left + width
        
    else:
        return lf
    
    new_lf = np.zeros(list(lf.shape[:2]) + [new_height, new_width] + list(lf.shape[4:]))
    new_lf[:,:,bound_up:bound_down, bound_left: bound_right, :] = lf
    return new_lf

aspect_ratio = desired_resolution[0]/desired_resolution[1]
padded_lf = pad_to_aspect_ratio(saved_lf, 1.25)
extra_10_dim = (int(padded_lf.shape[-2]*1.1), int(padded_lf.shape[-3]*1.1))
extra_10_lf = pad_to_shape(padded_lf, extra_10_dim)
resized_lf = resize_lightfield(extra_10_lf, desired_resolution)
transposed_lf = np.moveaxis(resized_lf, [0,1,2,3,4], [2,3,0,1,4])

(44, 43) (35, 35)


In [6]:
new_shape = list(transposed_lf.shape)[:2] + desired_views + [3]
result = np.zeros(new_shape, dtype= np.uint8)

for i, row in enumerate(tqdm.tqdm(transposed_lf)):
    for j, diffraction in enumerate(row):
        res = cv2.resize(diffraction, tuple(desired_views[::-1]), interpolation= cv2.INTER_LINEAR)
        result[i,j] = np.uint8(res)
        
result = np.moveaxis(result, [0,1,2,3,4], [2,3,0,1,4])

100%|██████████| 660/660 [01:07<00:00,  9.84it/s]


In [7]:
downsampled_result = result[::5, ::5] #can't display more than roughly 100x100 views, so just downsample by 3.
memory_size = result.nbytes / 1024**3 # in GB

print(result.shape, result.dtype, memory_size)
#DisplayLF(downsampled_result, width=572, height=440).show()

(144, 192, 660, 880, 3) uint8 44.86541748046875


In [8]:
datetime_str = datetime.datetime.today().strftime("%Y_%m_%d_%H_%M")
n_u, n_v, n_y, n_x, _ = result.shape
output_filename = Path(f"rendered_views_benton_{n_u}x{n_v}x{n_y}x{n_x}_" + datetime_str + ".npy")

print("Saving rendered lightfield at", output_filename, f" with size{memory_size:.02f}Gb")
#np.save(output_filename, result, allow_pickle= False)
print("Saved rendered lightfield in", output_filename)

#saved_lf = load_lightfield(output_filename)
saved_lf = result

# DisplayLF(saved_lf[::5, ::5]).show()

Saving rendered lightfield at rendered_views_benton_144x192x660x880_2021_06_04_18_16.npy  with size44.87Gb
Saved rendered lightfield in rendered_views_benton_144x192x660x880_2021_06_04_18_16.npy


In [9]:
rendered_lightfield = saved_lf
# rendered_lightfield = result
output_folder = Path(".") / "rendered_views" / str(output_filename)[:-4] # reuse .npy filename as folder name

def save_rendered_views(image_folder, rendered_lightfield):
    """
    Saves all the images of a lightfield in a folder - this one makes a folder for each line of images. 
    """
    image_folder.mkdir(parents= True, exist_ok=True)

    for row_idx, row in enumerate(tqdm.tqdm(rendered_lightfield)):
        row_folder = image_folder / f"{row_idx+1:03d}"
        row_folder.mkdir(parents= True, exist_ok=True)
        for column_idx, image_data in enumerate(row):
            if image_data.dtype != np.uint8:
                image_data = np.uint8(image_data*256)
                print("Converting np.float32 np.uint8. Use np.uint8 dtype to save rendered views")
            image = Image.fromarray(np.uint8(image_data))
            image.save(row_folder / f"{column_idx+1:04d}.bmp", format= "bmp", opt= "yes", quality= 75)

    print("Saved at", str(image_folder.absolute()))

save_rendered_views(output_folder, rendered_lightfield)

100%|██████████| 144/144 [26:27<00:00, 11.02s/it]

Saved at /mnt/WD14TB/felixh/code/holo_art_rendering/rendered_views/rendered_views_benton_144x192x660x880_2021_06_04_18_16


In [12]:
from boxsdk import DevelopmentClient, OAuth2, Client
from boxsdk.network.default_network import DefaultNetwork
import shutil


# Define client ID, client secret, and developer token.
CLIENT_ID = None
CLIENT_SECRET = None
ACCESS_TOKEN = None

# Read app info from text file
with open('box_credentials.txt', 'r') as app_cfg:
    """
    This is a .txt file with 3 lines
    CLIENT_ID
    CLIENT_SECRET
    ACCESS_TOKEN (the developer token)
    find at https://northwestern.app.box.com/developers/console/app/1484448/configuration 
    """
    CLIENT_ID = app_cfg.readline()
    CLIENT_SECRET = app_cfg.readline()
    ACCESS_TOKEN = app_cfg.readline()

oauth2 = OAuth2(CLIENT_ID, CLIENT_SECRET, access_token=ACCESS_TOKEN)

# Create the authenticated client
client = Client(oauth2)
root_folder = client.folder('0').get()

In [13]:
target_folder = None

for item in root_folder.get_items():
    print(item)
    if item.name=="LensCalibrated_6s":
        target_folder = item

<Box Folder - 132537575926 (20200227_ZWO_LaserDiode_250umStepsize_16bitTIFF)>
<Box Folder - 89719244986 (Lab2)>
<Box Folder - 90717384311 (Lab3)>
<Box Folder - 90852525215 (Lab4)>
<Box Folder - 123651585740 (LensCalibrated_6s)>
<Box Folder - 133978744306 (LensCalibration_50mm_F22_0.7mFocus)>
<Box Folder - 136593714079 (LF_043021_31by21)>
<Box Folder - 88717684651 (PhysLabs)>
<Box Folder - 137124462396 (Registered)>
<Box Folder - 136176191502 (Registered_Cropped)>
<Box File - 543373197663 (200gCartFan.cap)>
<Box File - 564856721296 (asimetric.csv)>
<Box File - 564835203299 (asimetric.xlsx)>
<Box File - 535412361488 (CIMG4595.MOV)>
<Box File - 535418148265 (CIMG4597.MOV)>
<Box File - 535800169673 (DataSheet.xlsx)>
<Box File - 535423343562 (Lab1 Vid.mp4)>
<Box File - 564841457870 (normal 1.csv)>
<Box File - 564845324545 (normal 1.xlsx)>
<Box File - 564836271331 (normal 2.csv)>
<Box File - 564857130879 (normal 2.txt)>
<Box File - 564853974345 (normal 2.xlsx)>
<Box File - 564835075293 (sime

In [14]:
#make a new folder for the rendered views
#output_folder = Path(".") / "rendered_views" / "rendered_views_benton_144x192x880x660_2021_06_04_13_51"
tmp_filename = "tmp_" + str(time.time()) + ".zip"
#upload zip to box
try:
    print("Creating temporary file")
    shutil.make_archive(tmp_filename[:-4], 'zip',output_folder)
    print("Uploading file", tmp_filename)
    target_folder.upload(tmp_filename, output_folder.name+".zip")
    print("Upload successful")
except:
    print("Upload failed")
    raise
finally:
    os.remove(tmp_filename)

Creating temporary file
Uploading file tmp_1622852916.7429585.zip


Request "POST https://upload.box.com/api/2.0/files/content" failed with ConnectionError exception: ConnectionError(ProtocolError('Connection aborted.', BrokenPipeError(32, 'Broken pipe')))


Upload failed


ConnectionError: ('Connection aborted.', BrokenPipeError(32, 'Broken pipe'))